In [1]:
# 用于将下载的csv文件合并、转换成h5文件

In [2]:
# 导入包
import pandas as pd
import numpy as np
import h5py
import multiprocessing
import os

In [3]:
# 参数设置
time_start = '201901' #时间格式，必须全为数字
time_end = '201912'
folder_path = "./Download" # 源数据
h5_file_name = 'NYC_2019.h5' # h5 文件名称
h5_path = './raw_bike_data/' # 目标路径
h5_file = h5_path + h5_file_name
multiprocessing.cpu_count()


16

In [16]:

# 设置文件夹路径
folder_path = "./Download"
# 使用os.walk()函数遍历文件夹中的所有文件
csv_files = []
for root, dirs, files in os.walk(folder_path):
    print('要合并的文件有:')
    for filename in files:
        # 打印文件路径
        year_month = filename[:6]
        if int(year_month) >= int(time_start) and int(year_month) <= int(time_end):
            file_path = os.path.join(root, filename)
            print(file_path)
            csv_files.append(file_path)

要合并的文件有:
./Download/201908-citibike-tripdata.csv
./Download/201905-citibike-tripdata.csv
./Download/201909-citibike-tripdata.csv
./Download/201907-citibike-tripdata.csv
./Download/201912-citibike-tripdata.csv
./Download/201901-citibike-tripdata.csv
./Download/201911-citibike-tripdata.csv
./Download/201903-citibike-tripdata.csv
./Download/201904-citibike-tripdata.csv
./Download/201910-citibike-tripdata.csv
./Download/201906-citibike-tripdata.csv
./Download/201902-citibike-tripdata.csv


In [6]:
# 多进程读取CSV文件

num_processes = multiprocessing.cpu_count()

def preprocess_nyc_bike(csv_file):
    df = pd.read_csv(csv_file)
    drop_columns = ['tripduration','start station id','start station name','end station id','end station name','bikeid','usertype','birth year','gender']
    df.drop(drop_columns,axis=1,inplace=True)
    df.columns = ['stime','etime','slat','slon','elat','elon']
    return df

with multiprocessing.Pool(num_processes) as pool:
    dfs = pool.map(preprocess_nyc_bike, csv_files)

In [7]:
dfs

[                            stime                     etime       slat  \
 0        2019-07-01 00:00:00.1320  2019-07-01 00:14:58.0040  40.756800   
 1        2019-07-01 00:00:05.1780  2019-07-01 00:04:32.4500  40.776321   
 2        2019-07-01 00:00:05.2130  2019-07-01 00:36:46.7490  40.724537   
 3        2019-07-01 00:00:08.6010  2019-07-01 00:27:48.8050  40.718710   
 4        2019-07-01 00:00:12.1580  2019-07-01 00:02:01.5670  40.805973   
 ...                           ...                       ...        ...   
 2181059  2019-07-31 23:59:45.2950  2019-08-01 00:10:29.8640  40.686919   
 2181060  2019-07-31 23:59:52.7170  2019-08-01 00:09:42.0870  40.788390   
 2181061  2019-07-31 23:59:52.8280  2019-08-01 00:21:35.1090  40.711731   
 2181062  2019-07-31 23:59:57.1130  2019-08-01 00:05:50.1350  40.750585   
 2181063  2019-07-31 23:59:57.4870  2019-08-01 00:04:32.2070  40.757632   
 
               slon       elat       elon  
 0       -73.982912  40.754557 -73.965930  
 1       -

In [8]:
# 合并dataframe并写入h5文件
df = pd.concat(dfs)
df.to_csv('data/NYC_bike/raw_bike_data/NYC_Bike.csv',index=False)